# Authentication and Core Libraries (Google Earth Engine + geemap)

## Libraries and datasets (quick reference)
1. This notebook uses Earth Engine and geemap for interactive maps.
2. Dataset: Landsat 9 surface reflectance (`LANDSAT/LC09/C02/T1_L2`).



In [2]:
# Install the libraries we need for this notebook.
# We use !pip so it works in both Colab and local Jupyter kernels.
!pip install -q --upgrade earthengine-api geemap matplotlib numpy pandas rasterio


In [3]:
# Import Earth Engine so we can access datasets and run geospatial analysis.
import ee  # Earth Engine Python API

# Import geemap for interactive maps inside notebooks.
import geemap  # Map display helper for Earth Engine

# Try to initialize Earth Engine. If it fails, run authentication first.
# This works in both Colab and local Jupyter.
try:
    ee.Initialize()  # Connect to Earth Engine using saved credentials
except Exception:
    ee.Authenticate()  # Open a browser-based login flow
    ee.Initialize()  # Retry initialization after authentication

# Create a map so we can visualize results.
Map = geemap.Map()  # Interactive map widget

# Confirm that Earth Engine initialized successfully.
print('Earth Engine initialized.')


Earth Engine initialized.


## Landsat 9 quick look

We will pull the most recent Landsat 9 image over Stanford University and visualize it. This demonstrates how to filter an ImageCollection by location and time.


In [4]:
# Define a point representing Stanford University (longitude, latitude).
stanford_point = ee.Geometry.Point([-122.1697, 37.4275])  # Point geometry

# Load the Landsat 9 surface reflectance image collection.
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')  # Landsat 9 dataset

# Filter to the most recent image over Stanford.
image = (
    collection
    .filterBounds(stanford_point)  # Keep images that cover the point
    .sort('system:time_start', False)  # Sort newest to oldest
    .first()  # Take the newest image
)

# Center the map on the image and add a false-color layer.
Map.centerObject(image, zoom=10)  # Zoom to the area
Map.addLayer(
    image,
    {'min': 0, 'max': 30000, 'bands': ['SR_B5', 'SR_B4', 'SR_B1']},
    'Latest Landsat 9 (Stanford)'
)

# Display the map in the notebook.
Map


Map(center=[37.47136588057848, -122.13773874948753], controls=(WidgetControl(options=['position', 'transparent…

## Vegetation example (NDVI)

NDVI is a simple index that highlights vegetation. Higher values often indicate healthier or denser vegetation.


In [5]:
# Calculate NDVI (Normalized Difference Vegetation Index).
# NDVI = (NIR - Red) / (NIR + Red)
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4'])  # Band 5 = NIR, Band 4 = Red

# Visualize NDVI using a color palette.
ndvi_vis = {
    'min': -0.1,
    'max': 0.8,
    'palette': ['blue', 'white', 'green']
}

# Add NDVI to the map.
Map.addLayer(ndvi, ndvi_vis, 'NDVI')

# Display the updated map.
Map


Map(bottom=25607.0, center=[37.87740035478434, -121.28692770571408], controls=(WidgetControl(options=['positio…